In [1]:
import pandas as pd
from gpt_sender_async import GPT_API as gpt
import asyncio
from tqdm import tqdm
import json

In [3]:
df = pd.read_csv("train_data.csv", index_col=0)

In [4]:
df.head(2)

,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер
index,,,,,
0,Не работает блок питания,В течении недели перестал внезапно работать бл...,Ноутбук,Блок питания,C223100360
1,Не включается SILA LLC HK2-3404.,Добрый день!_x000D_\nПодскажите пожалуйста что...,Ноутбук,Материнская плата,C223094534


In [15]:
values = []
for value in df["Точка отказа"].value_counts().index:
    values.append(value.strip())
values = set(values)

In [83]:
res = ""
for value in values:
    res += f'{{{{"{value}: bool}}}}, ' 

In [98]:
def get_prompt(tema, opisanie, oborudovanie):
    return '''"""
Тема: {}
Описание: {}
Оборудование: {}
"""
Выше представлено обращение пользователя в тех поддержку с темой, описанием и оборудованием.
Ты должна выполнить multilable classification, определить к каким классам точки отказа относится обращения, классов может быть

Верни в формате json. 
Пример: {{{{"Матрица: bool}}, {{"Клавиатура: bool}}, {{"Оперативная память: bool}}, {{"Блок питания: bool}}, {{"Материнская плата: bool}}, {{"Сервер: bool}}, {{"Диск: bool}}, {{"Wi-fi модуль: bool}}, {{"Корпус: bool}}, {{"Wi-fi антенна: bool}}, {{"Jack: bool}}, {{"SFP модуль: bool}}, {{"Камера: bool}}, {{"Динамики: bool}}, {{"Консультация: bool}}, {{"Вентилятор: bool}}, {{"Аккумулятор: bool}}, {{"Программное обеспечение: bool}}}} 
Где вместо bool должно быть true если проблема связана с этим классом и false если нет.
"Консультация" - если у пользователя есть вопрос не касательно поломки (в какой филиал обратиться или еще подобные вопросы)
"Jack" - если сломан разъем для наушников
Return only JSON.
'''.format(tema, opisanie, oborudovanie)
    

In [100]:
async def get_responses(prompts):
    tasks = [gpt.get_response(prompt, json_output=True) 
             for prompt in prompts]
    responses = await asyncio.gather(*tasks)
    return responses

In [103]:
batch_size = 8
result = []
for i in tqdm(range(0, len(df), batch_size)):
    # Add the batch to the batches list.

    data = []
    for index, row in df[i:i+batch_size].iterrows():
        data.append([row["Тема"], row["Описание"], row["Тип оборудования"]])

    summaries = await get_responses([get_prompt(*row) for row in data])

    for row, summary in zip(data, summaries):

        if summary is None:
            continue
        
        result.append({"Тема": row[0], "Описание": row[1], "Тип оборудования": row[2], "Обращения": summary})

100%|██████████| 25/25 [01:51<00:00,  4.48s/it]


In [106]:
with open("multilable_gpt4o.jsonl", 'w') as file:
    for entry in result:
        json_line = json.dumps(entry,ensure_ascii=False)  # Convert dictionary to JSON string
        file.write(json_line + '\n')   